# Introduction to `GeoGraph` - 4 March 2021

---

Recent progress:
- First results in classification (WS1) (Show video on WS1)
  - Additional work: IR, UNet
- Fragmentation Baseline completed (Show summary notebook)
- Ongoing: building GeoGraph platform (WS3&4)
  - Completed: loading & accessing, habitats, basic visualisation, identification
  - Ongoing: timeline, advanced visualisation, metrics

## 0. Setup and Loading package

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
from src.data_loading import jasmin
from src.models import geograph, visualisation
from src.constants import PREFERRED_CRS

---

## 1. Loading Data

In [ ]:
# Loading Polesia data set from shared Jasmin folder
gdf = jasmin.get_polesia_data()

# Looking at south-west corner of data
# Choosen because are of wilderness as discussed
minx, miny, maxx, maxy = gdf.total_bounds
square_len = 25000

gdf = gdf.cx[ minx:minx+square_len  ,  miny:miny+square_len]

print("Number of patches in region of interest:", len(gdf))
gdf.head(5)

---

## 2. Creating `GeoGraph`

In [ ]:
# Building the main graph structure

graph = geograph.GeoGraph(gdf, crs=PREFERRED_CRS, columns_to_rename={"Eunis_name":"class_label","AREA":"area"})

---

## 3. Creating Habitats

In [ ]:
# First selecting the classes that make up our habitat
# We chose all classes with 'pine' in the name.
pine_classes = [label for label in graph.df.class_label.unique() if 'pine' in label]
pine_classes

In [ ]:
# Proposed distance from email: mobile (<100m), semi mobile (<25m) and sessile (<5m)
graph.add_habitat('Sessile (5m) - pine', 
                  max_travel_distance=5, 
                  valid_classes=pine_classes)

graph.add_habitat('Semi mobile (25m) - pine', 
                  max_travel_distance=25, 
                  valid_classes=pine_classes)

graph.add_habitat('Mobile (200m) - pine',
                  max_travel_distance=200, 
                  valid_classes=pine_classes)

---

## 4. Graph Interaction

In [ ]:
viewer = visualisation.GeoGraphViewer()
viewer.add_graph(graph, name='Polesia data')
viewer.add_widgets()
viewer.add_hover_widget()
viewer

### Things to consider in GeoGraphViewer
- How `max_travel_distance` affects the number of components in habitat